<a href="https://colab.research.google.com/github/Nikhithaprasannadurga/FMML-Labs/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [3]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [4]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [5]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [6]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [7]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [8]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [9]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [10]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [11]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [12]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [13]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [14]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
The accuracy of the validation set can be affected by increasing or reducing the percentage of data allocated to the validation set in the following ways:

1. Increasing the percentage of the validation set:

Positive Effects:

Better generalization estimate: With a larger validation set, the accuracy tends to represent the model's performance more reliably, as it evaluates the model on a broader and more diverse set of data points.

Reduced variance in results: A larger sample size reduces the influence of outliers and random fluctuations in the accuracy metric.


Negative Effects:

Reduced training data: Allocating more data to validation means less data is available for training, which can hurt the model's ability to learn complex patterns, particularly for smaller datasets. This may result in a lower-quality model, leading to reduced validation accuracy.



2. Reducing the percentage of the validation set:

Positive Effects:

More data for training: With more data available for training, the model can potentially learn better, improving its ability to generalize.


Negative Effects:

Unreliable evaluation: A smaller validation set may not adequately represent the diversity of the dataset, leading to a less reliable estimation of the model's performance.

Higher variance: The accuracy metric on a small validation set can vary significantly depending on the specific data points included, making it harder to trust.



Key Considerations:

Dataset size: The impact depends heavily on the overall dataset size. For small datasets, reducing the validation set size may give the model more training data but risks overfitting or unreliable validation metrics. For large datasets, the impact is generally less pronounced.

Cross-validation: To mitigate these trade-offs, techniques like k-fold cross-validation can be used. This method ensures the model is evaluated on all data points without reducing the training data unnecessarily.


In summary:

A larger validation set gives a more reliable performance estimate but may reduce model quality due to less training data.

A smaller validation set allows more training data but may result in unreliable evaluation metrics.



2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

The size of the train and validation sets directly influences how well the validation set's accuracy predicts the test set's accuracy. Here's how:


---

1. Size of the Training Set

Larger Training Set:

Positive Effect: A larger training set allows the model to learn more comprehensive patterns and generalize better, leading to a more reliable validation accuracy.

Test Set Prediction: If the model is well-trained, the validation accuracy is more likely to reflect the test accuracy accurately.


Smaller Training Set:

Negative Effect: A smaller training set may result in underfitting, as the model doesn't have enough data to learn effectively. This can cause the validation accuracy to deviate significantly from the test accuracy.

Test Set Prediction: Validation accuracy becomes a poor predictor of test accuracy due to an undertrained model.




---

2. Size of the Validation Set

Larger Validation Set:

Positive Effect: A larger validation set reduces variance and better represents the dataset's true distribution, making the validation accuracy a more reliable predictor of test accuracy.

Negative Effect: However, this comes at the cost of a smaller training set, which may reduce the model's ability to generalize effectively (especially for small datasets).


Smaller Validation Set:

Positive Effect: Allocating more data to training allows the model to learn better, which could improve test performance.

Negative Effect: A small validation set increases the risk of overfitting to the specific validation data and may not reflect the dataset's diversity. This leads to validation accuracy being a less reliable predictor of test accuracy.




---

Key Factors Affecting the Relationship

1. Dataset Size:

For large datasets, you can afford to allocate more data to both training and validation, leading to reliable validation accuracy and good generalization to the test set.

For small datasets, you must carefully balance the split. Too much validation data reduces training effectiveness, while too little leads to unreliable validation metrics.



2. Model Complexity:

Complex models trained on small training sets are more likely to overfit, making validation accuracy less representative of test accuracy.

Simpler models are more robust to such issues.



3. Validation-Set Bias:

If the validation set is not representative of the test set (e.g., due to poor sampling), validation accuracy will not reliably predict test accuracy, regardless of size.



4. Cross-Validation:

Using techniques like k-fold cross-validation provides a more robust estimate of test accuracy by averaging performance across multiple splits.





---

Summary

A balance between training and validation sizes is crucial for validation accuracy to be a good predictor of test accuracy.

A larger training set improves the model's generalization, while a larger validation set ensures the evaluation is reliable.

For small datasets, techniques like cross-validation are essential to avoid trade-offs between training and validation data.



3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

A good percentage to reserve for the validation set depends on the size of the dataset and the complexity of the model. Here are some general guidelines:


---

General Recommendation:

Common Practice:

Reserve 20% of the dataset for validation. This is a standard split that works well for many cases where the dataset size is moderate or large.

If you also have a separate test set, you might use 10-15% for validation to maximize training data.




---

Dataset-Specific Recommendations:

1. Small Datasets (<10,000 samples):

Validation Set: 10-20%
Use a smaller percentage (10-15%) to retain more data for training, but rely on techniques like k-fold cross-validation to evaluate the model robustly.



2. Medium Datasets (10,000-100,000 samples):

Validation Set: 15-20%
This range provides a balance between enough training data to optimize the model and enough validation data for reliable evaluation.



3. Large Datasets (>100,000 samples):

Validation Set: 5-10%
A smaller fraction is sufficient since even a small percentage represents a large number of samples, providing robust validation metrics.





---

Model-Specific Considerations:

1. Simple Models (e.g., linear regression):

Smaller validation sets (e.g., 10-15%) are fine since simpler models don't require as much data for reliable evaluation.



2. Complex Models (e.g., deep neural networks):

Larger validation sets (e.g., 15-20%) are better, as they need more diverse samples to provide an accurate performance estimate.





---

When to Adjust the Percentage:

1. Increase Validation Size If:

You observe high variance in validation metrics across epochs or folds.

Validation accuracy doesn't align well with test accuracy, suggesting unreliable validation evaluation.



2. Decrease Validation Size If:

The training set is too small, causing underfitting.

Cross-validation or other techniques can compensate for a smaller validation set.





---

Key Takeaway:

Default Split: Start with 80% training, 20% validation for most datasets.

Adjust based on dataset size, model complexity, and availability of a separate test set.

For small datasets, prioritize cross-validation over static splits to balance training and validation needs effectively.




Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [15]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [16]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits typically provides more consistent and reliable results. This approach, often used in cross-validation, reduces the impact of randomness and ensures that the evaluation better reflects the model's ability to generalize. Here's why:


---

Benefits of Averaging Validation Accuracy Across Splits:

1. Reduces Variance:

A single validation split may not be representative of the entire dataset due to random variations or an unbalanced split.

Averaging over multiple splits smooths out these variations, resulting in a more consistent and reliable estimate of performance.



2. Accounts for Data Distribution:

By evaluating on different subsets of the data, you ensure the model is tested on a broader variety of data points. This reduces bias caused by an unrepresentative validation set.



3. Improves Generalization Estimate:

Averaging validation results across splits gives a better approximation of how the model will perform on unseen data (like the test set).



4. Minimizes Overfitting to Validation Set:

When you repeatedly train and validate on different splits, the model's performance isn't overly dependent on a specific validation set, reducing the risk of overfitting to that set.





---

When Is Averaging Useful?

1. Small Datasets:

Particularly effective for small datasets where random variations in a single split can have a significant impact.

Techniques like k-fold cross-validation are ideal.



2. Unbalanced Datasets:

Ensures that all subsets of the data (including minority classes) are included in both training and validation, leading to fairer evaluation.





---

How to Average Validation Accuracy Across Splits:

1. k-Fold Cross-Validation:

Split the dataset into k equally sized subsets (folds).

Train the model on k-1 folds and validate on the remaining fold, repeating this process k times.

Average the validation accuracies across the k folds.



2. Repeated Random Splits (Monte Carlo Cross-Validation):

Randomly split the dataset into training and validation sets multiple times and average the validation results.





---

Limitations:

Increased Computational Cost:

Training the model multiple times for different splits can be computationally expensive, especially for large datasets or complex models.


Dataset Size Dependency:

For extremely small datasets, even averaging might not fully address issues like data scarcity or imbalance.




---

Key Takeaway:

Yes, averaging validation accuracy across multiple splits is a highly effective way to obtain consistent and reliable performance estimates, especially when combined with methods like k-fold cross-validation. It provides a robust measure of how well the model is likely to generalize to unseen data.


2. Does it give more accurate estimate of test accuracy?

Yes, averaging validation accuracy across multiple splits generally provides a more accurate estimate of test accuracy compared to using a single validation split. Here's why:


---

1. Reduces the Impact of Random Splits

A single validation split may not be representative of the test data distribution, especially if the dataset is small or imbalanced.

Averaging across multiple splits ensures that the evaluation is based on different subsets of the data, making the results less sensitive to the specific choice of the validation set.



---

2. More Representative Evaluation

Broader Data Coverage:
By using multiple splits, the model is trained and validated on more diverse subsets of the dataset, capturing a wider range of patterns and edge cases. This helps align validation performance more closely with test performance.

Reduced Sampling Bias:
Any accidental biases in a single validation split are averaged out across multiple splits.



---

3. Handles Overfitting Risks

When evaluating on a single validation set, there's a risk of the model overfitting to the specific validation data, leading to an overly optimistic performance estimate.

Multiple splits mitigate this risk by preventing the model from repeatedly seeing the same validation data during training, leading to a more realistic test accuracy estimate.



---

4. Consistency with Test Data Distribution

If the test data comes from the same distribution as the dataset used for training/validation, the averaged validation accuracy from multiple splits is more likely to approximate the true test accuracy.



---

Limitations to Consider

Not Perfect: While averaging validation accuracy provides a better estimate, it doesn't guarantee an exact match with test accuracy. Test accuracy can still deviate due to:

Differences between test and training/validation distributions (data drift).

Hyperparameter tuning or design choices inadvertently overfitting to validation performance.


Computational Cost: Cross-validation or repeated splits are computationally intensive since the model must be trained multiple times.



---

When Does It Work Best?

Small to Medium Datasets: Averaging is especially beneficial for smaller datasets, where the test accuracy might otherwise be misestimated due to random variations in the validation set.

Balanced Datasets: If the data distribution is similar across splits and to the test set, averaging is highly effective.

Cross-Validation Techniques: Techniques like k-fold cross-validation ensure systematic use of the entire dataset for training and validation, enhancing the reliability of the test accuracy estimate.



---

Key Takeaway

Averaging validation accuracy across multiple splits does provide a more accurate and reliable estimate of test accuracy, especially when combined with cross-validation techniques. However, its effectiveness depends on the dataset size, distribution, and similarity to the test set.


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

Increasing the number of iterations (e.g., in cross-validation or repeated random splits) generally improves the reliability and accuracy of the test accuracy estimate, but the benefits diminish as the number of iterations becomes very large. Here's a detailed breakdown:


---

Effect of Number of Iterations:

1. More Iterations Provide Better Estimates (Up to a Point):

Reduced Variance: Each iteration gives an independent estimate of validation accuracy. With more iterations, the final average becomes more stable, reducing the variability of the estimate. This leads to a more consistent and reliable approximation of the test accuracy.

Broader Data Coverage: More iterations ensure the model is validated on a greater variety of data splits, making the evaluation more representative of the overall dataset.



2. Law of Diminishing Returns:

After a certain number of iterations, additional iterations provide only marginal improvement. The estimate's accuracy becomes limited by factors such as:

The inherent variability in the dataset.

The similarity between the training/validation splits and the test set.


Beyond this point, computational cost increases without significant gains in accuracy.



3. Improved Confidence Intervals:

With higher iterations, you can compute narrower confidence intervals for the validation accuracy, providing a better sense of how close the estimate is to the true test accuracy.





---

Trade-offs of Increasing Iterations:

1. Positive Effects:

More iterations reduce the likelihood of biased results caused by outlier splits.

The estimate becomes less sensitive to random variations in how the data is split.



2. Negative Effects:

Computational Cost: Increasing iterations requires retraining the model multiple times, which can be time-consuming and computationally expensive, particularly for complex models or large datasets.

Diminished Returns: Beyond a certain point, additional iterations have negligible impact on the estimate's accuracy.





---

Guidelines for Choosing the Number of Iterations:

1. k-Fold Cross-Validation:

A typical choice is 5-fold or 10-fold cross-validation, which provides a good balance between computational cost and accuracy of the estimate.

For smaller datasets, you may consider using leave-one-out cross-validation (LOOCV) to maximize the data used for training.



2. Repeated Cross-Validation:

If computational resources allow, you can repeat k-fold cross-validation (e.g., 3-10 times) and average the results for even more robust estimates.



3. Monte Carlo Validation:

Repeatedly split the data into random training and validation sets. 30-50 iterations are often sufficient to get a reliable estimate, though the exact number depends on the dataset size and variability.





---

Key Takeaway:

Higher iterations generally lead to better estimates of test accuracy by reducing variance and increasing reliability.

However, after a certain point, the benefits diminish, and the computational cost outweighs the marginal gains.

A practical number of iterations depends on the dataset size, model complexity, and available computational resources. For most cases, 5-10 k-fold cross-validation or 30-50 Monte Carlo iterations provides a robust estimate without excessive overhead.



4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Yes, increasing the number of iterations (e.g., via techniques like repeated cross-validation) can help mitigate the challenges posed by a very small training or validation dataset. However, it is not a perfect solution and has limitations. Here’s how it helps and where it falls short:


---

How Increasing Iterations Helps with Small Datasets:

1. Maximizes Data Usage:

In cross-validation, every data point gets used for both training and validation across different iterations. This ensures that even with a small dataset, all data contributes to model training and evaluation at some point.



2. Improves Estimate Stability:

Repeating iterations averages out random noise or bias caused by specific training or validation splits, leading to a more reliable estimate of model performance.



3. Reduces Overfitting to Validation Set:

With more iterations, the model is validated on a variety of splits, reducing the chance that the validation performance is overly dependent on a specific subset of data.



4. Addresses Limited Validation Data:

A small validation dataset might not represent the overall distribution well. By increasing iterations, you validate the model on different subsets, creating a more comprehensive evaluation.





---

Limitations of Increasing Iterations for Small Datasets:

1. Does Not Create New Data:

Repeatedly splitting a small dataset cannot compensate for the lack of diversity or representativeness in the dataset itself. If the data is inherently limited or imbalanced, even many iterations won’t overcome this issue.



2. Training Data Limitation:

In cross-validation, the training set remains small in each iteration (e.g., in 10-fold cross-validation, 90% of the data is used for training). Increasing iterations does not address the fundamental limitation of having insufficient data for learning complex patterns.



3. Overfitting Risk in Small Datasets:

A very small dataset increases the likelihood of overfitting during training, especially with high-capacity models. Cross-validation helps assess overfitting but doesn’t prevent it.



4. Computational Cost:

Small datasets often require complex models to achieve good performance, making repeated training and validation computationally expensive.





---

Strategies to Complement Iteration Increases:

1. Data Augmentation (for small training datasets):

For tasks like image or text classification, generate synthetic data through augmentation techniques to artificially increase the size of the training set.



2. Simpler Models:

Use simpler models or regularization techniques to reduce overfitting when training on small datasets.



3. Transfer Learning:

Pretrained models can leverage knowledge from a larger dataset and adapt to the small dataset, reducing the reliance on extensive training data.



4. Bootstrapping (for small validation datasets):

Instead of increasing iterations, use bootstrap sampling to repeatedly evaluate the model on random subsets of the validation data, creating a distribution of performance estimates.



5. Bayesian Methods:

Bayesian approaches can incorporate uncertainty into predictions, which is particularly useful for small datasets.





---

Key Takeaway:

Increasing iterations can help extract maximum value from a small training or validation dataset by providing more robust and stable performance estimates. However, it does not solve fundamental issues like lack of diversity, imbalance, or insufficient data for learning complex patterns. Combining increased iterations with complementary techniques (e.g., data augmentation, transfer learning, or simpler models) is the most effective approach for dealing with small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.